---

* 출처: LangChain 공식 문서 또는 해당 교재명
* 원본 URL: https://smith.langchain.com/hub/teddynote/summary-stuff-documents

---

### **5. `05. LLM-as-Judge`**

#### **2) `Question-Answer Evaluator`**

* 가장 기본 기능을 가진 `Evaluator` = `Query` - `Answer` 평가하기

* 사용자 입력 = `input` → `LLM`이 생성한 답변 → `prediction`으로 정답 답변은 `reference`로 정의됨

  * `Prompt`변수 
    * `query`: 질문
    * `result`: `LLM` 답변
    * `answer`: 정답 답변

* **새로운 가상환경 생성 - `lc_eval_env`**

  * `Python-3.12`
  * `Pydantic`: `ver 1.10.18`

In [ ]:
import pydantic
print(f"Pydantic 버전: {pydantic.__version__}")

<small>

* **`Pydantic 버전: 1.10.18`** - (`0.1s`)

---

* **`환경 설정`**

In [ ]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()                           # True

In [ ]:
from langsmith import Client
from langsmith import traceable

import os

# LangSmith 환경 변수 확인

print("\n--- LangSmith 환경 변수 확인 ---")
langchain_tracing_v2 = os.getenv('LANGCHAIN_TRACING_V2')
langchain_project = os.getenv('LANGCHAIN_PROJECT')
langchain_api_key_status = "설정됨" if os.getenv('LANGCHAIN_API_KEY') else "설정되지 않음" # API 키 값은 직접 출력하지 않음

if langchain_tracing_v2 == "true" and os.getenv('LANGCHAIN_API_KEY') and langchain_project:
    print(f"✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='{langchain_tracing_v2}')")
    print(f"✅ LangSmith 프로젝트: '{langchain_project}'")
    print(f"✅ LangSmith API Key: {langchain_api_key_status}")
    print("  -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.")
else:
    print("❌ LangSmith 추적이 완전히 활성화되지 않았습니다. 다음을 확인하세요:")
    if langchain_tracing_v2 != "true":
        print(f"  - LANGCHAIN_TRACING_V2가 'true'로 설정되어 있지 않습니다 (현재: '{langchain_tracing_v2}').")
    if not os.getenv('LANGCHAIN_API_KEY'):
        print("  - LANGCHAIN_API_KEY가 설정되어 있지 않습니다.")
    if not langchain_project:
        print("  - LANGCHAIN_PROJECT가 설정되어 있지 않습니다.")

<small>

* 셀 출력

    ```bash
    --- LangSmith 환경 변수 확인 ---
    ✅ LangSmith 추적 활성화됨 (LANGCHAIN_TRACING_V2='true')
    ✅ LangSmith 프로젝트: 'LangChain-prantice'
    ✅ LangSmith API Key: 설정됨
    -> 이제 LangSmith 대시보드에서 이 프로젝트를 확인해 보세요.
    ```

In [ ]:
import os
from myrag import PDFRAG                        # local 임베딩 버전으로 수정한 myrag.py 불러오기
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI

# API 키 확인
from dotenv import load_dotenv

GOOGLE_API_KEY=os.getenv("GOOGLE_API_KEY2")

if not os.getenv("GOOGLE_API_KEY2"):
    os.environ["GOOGLE_API_KEY2"] = input("Enter your GOOGLE_API_KEY2: ")

if "GOOGLE_API_KEY2" not in os.environ:
    print("❌ 경고: GOOGLE_API_KEY2 환경 변수가 설정되지 않았습니다. 반드시 설정해야 gemini LLM이 작동합니다.")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm2 = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0, api_key=os.getenv("GOOGLE_API_KEY2"))

print("✅ gemini-2.5-flash-lite 성공!")

<small>

* **`✅ 성공!`** 

  * *`API 할당량 부족으로 두번쨰 계정으로 다시 시도`*

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm2 = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite", temperature=0, api_key=os.getenv("GOOGLE_API_KEY2"))

rag = PDFRAG(
    "../15_Evaluations/data/SPRI_AI_Brief_2023년12월호_F.pdf",
    llm2,
)

print("✅ PDFRAG 생성 성공!")

<small>

* **`PDFRAG 생성`** - (`4.9s`)

    ```markdown

    ✅ 문서 로드 완료: 23개 페이지
    ✅ 문서 분할 완료: 43개 청크
    ✅ 임베딩 모델 로드: all-MiniLM-L6-v2
    ✅ 벡터스토어 생성 완료
    ✅ PDFRAG 생성 성공!

    ```

In [ ]:
# 검색기(retriever) 생성
retriever = rag.create_retriever()

<small>

* **`✅ 검색기 생성 완료 (k=4)`**

In [ ]:
# 체인(chain) 생성
chain = rag.create_chain(retriever)

<small>

* **`✅ RAG 체인 생성 완료`**

In [ ]:
# 질문
answer = chain.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")
print(answer)

<small>

* **`삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다.`** - (`2.0s`)

In [ ]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain.invoke(inputs["question"])}

# 사용자 질문 예시
llm_answer = ask_question(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)

llm_answer

<small>

* **`ask_question()`** - (`1.2s`)

    ```python

        {'answer': "삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."}

    ```

In [ ]:
# evaluator prompt 출력을 위한 함수
def print_evaluator_prompt(evaluator):
    return evaluator.evaluator.prompt.pretty_print()

In [ ]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain_google_genai import ChatGoogleGenerativeAI

# qa 평가자 생성
qa_evalulator = LangChainStringEvaluator(
    "qa",
    config={
        "llm":ChatGoogleGenerativeAI(
            model="gemini-2.5-flash-lite", 
            temperature=0, 
            google_api_key=os.getenv("GOOGLE_API_KEY2"))
        }
)

# 프롬프트 출력
print_evaluator_prompt(qa_evalulator)

<small>

* **`qa_evaluator`** - (`0.1s`)

    ```markdown

    You are a teacher grading a quiz.
    You are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.

    Example Format:
    QUESTION: question here
    STUDENT ANSWER: student's answer here
    TRUE ANSWER: true answer here
    GRADE: CORRECT or INCORRECT here

    Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

    QUESTION: *{query}*
    STUDENT ANSWER: *{result}*
    TRUE ANSWER: *{answer}*
    GRADE:

    ```

* **`평가 진행` → 출력한 `URL` 이동 → `결과 확인하기`**

In [ ]:
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[qa_evalulator],
    experiment_prefix="RAG_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "QA Evaluator 를 활용한 평가",
    },
)

<small>

* 결과 확인하기

  * `RAG_EVAL` / *`COMPILED`* - (*`45s`*)
  * ![`RAG_EVAL_COMPILED`](../15_Evaluations/assets/RAG_EVAL_1.png)

  * 

  * `RAG_EVAL` / *`CORRECT=1`, `INCORRECT=4`* - (*`7.8s`*)
  * ![`RAG_EVAL_MORE_INCORRECT`](../15_Evaluations/assets/RAG_EVAL_2.png)

---

* **`RAG`** 시스템 개선하기

  * **a. `Retriever` 개선하기**
  * **b. `Chunk Size` 조정하기**

In [ ]:
# ========================================
# Chunk Size 조정
# ========================================
from myrag2 import PDFRAG
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv

load_dotenv() 

# Chunk Size 조정
rag2 = PDFRAG(
    "../15_Evaluations/data/SPRI_AI_Brief_2023년12월호_F.pdf",
    ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-lite", 
        temperature=0, 
        google_api_key=os.getenv("GOOGLE_API_KEY2")),
)

<small>

* **`rag2 생성`** - (`12.2s`)

    ```mardkwon

    ✅ 문서 로드 완료: 23개 페이지
    ✅ 문서 분할 완료: 72개 청크
    ✅ 임베딩 모델 로드: all-MiniLM-L6-v2
    ✅ 벡터스토어 생성 완료

    ```

In [ ]:
# ========================================
# Top-K 증가
# ========================================

# 개선
retriever2 = rag2.create_retriever()        # myrag2.py 불러오기

# 체인 재생성
chain2 = rag2.create_chain(retriever2)

<small>

* **`✅ 검색기 생성 완료`**

* **`✅ RAG 체인 생성 완료`**

In [ ]:
# 질문에 대한 답변 생성
chain2.invoke("삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?")

<small>

* **`'삼성전자가 자체 개발한 생성형 AI의 이름은 제공된 문서에서 찾을 수 없습니다.'`** - (`1.4s`)

In [ ]:
# 질문에 대한 답변하는 함수를 생성
def ask_question(inputs: dict):
    return {"answer": chain2.invoke(inputs["question"])}

# 사용자 질문 예시
llm_answer = ask_question(
    {"question": "삼성전자가 자체 개발한 생성형 AI의 이름은 무엇인가요?"}
)

llm_answer

<small>

* **`ask_question()`** - (`1.3s`)

    ```python

    {'answer': '삼성전자가 자체 개발한 생성형 AI의 이름은 제공된 문서에서 찾을 수 없습니다.'}

    ```

In [ ]:
# evaluator prompt 출력을 위한 함수
def print_evaluator_prompt(evaluator):
    return evaluator.evaluator.prompt.pretty_print()

In [ ]:
from langsmith.evaluation import evaluate, LangChainStringEvaluator
from langchain_google_genai import ChatGoogleGenerativeAI


# qa 평가자 생성
qa_evalulator = LangChainStringEvaluator(
    "qa",
    config={
        "llm":ChatGoogleGenerativeAI(
            model="gemini-2.5-flash-lite", 
            temperature=0, 
            google_api_key=os.getenv("GOOGLE_API_KEY2"))
        }
)


# 프롬프트 출력
print_evaluator_prompt(qa_evalulator)

<small>

* **`qa_evaluator`**

    ```markdown

    You are a teacher grading a quiz.
    You are given a question, the student's answer, and the true answer, and are asked to score the student answer as either CORRECT or INCORRECT.

    Example Format:
    QUESTION: question here
    STUDENT ANSWER: student's answer here
    TRUE ANSWER: true answer here
    GRADE: CORRECT or INCORRECT here

    Grade the student answers based ONLY on their factual accuracy. Ignore differences in punctuation and phrasing between the student answer and true answer. It is OK if the student answer contains more information than the true answer, as long as it does not contain any conflicting statements. Begin! 

    QUESTION: *{query}*
    STUDENT ANSWER: *{result}*
    TRUE ANSWER: *{answer}*
    GRADE:

    ```

In [ ]:
dataset_name = "RAG_EVAL_DATASET"

# 평가 실행
experiment_results = evaluate(
    ask_question,
    data=dataset_name,
    evaluators=[qa_evalulator],
    experiment_prefix="RAG_EVAL_2",
    # 실험 메타데이터 지정
    metadata={
        "variant": "myrag2.py 반영",
    },
)

<small>

* **개선 시도: `RAG_EVAL_2`**

  ```bash

  View the evaluation results for experiment: 'RAG_EVAL_2-00c422bd' at:
  https://smith.langchain.com/o/2c3342d3-1170-4ffa-86fd-f621199e0b9c/datasets/420dd308-2ebd-44c9-8ce8-9aff3886dc8e/compare?selectedSessions=cfb85525-ad18-4c76-9e47-4b2c1beb95fa


  0it [00:00, ?it/s]

  5it [00:04,  1.12it/s]

  ```

* 결과 확인하기

<br>

* 
  * `RAG_EVAL` / *`CORRECT=2`, `INCORRECT=3`* - (*`7.3s`*)
  * ![`RAG_EVAL_MORE_INCORRECT`](../15_Evaluations/assets/RAG_EVAL_3.png)

---

* **`RAG`** 시스템 개선하기_2

  * **c. `chunk_size` 조정**
  * **d. 잦은 커널 충돌 → `python` 파일로 실행하기**

* c.

  * `chunck_size` 조절 test
    * **`chunk_size` 지정 ❌** or **`chunk_size` = `4` → 검색 결과 없음**
      * *`'죄송합니다. 제공된 문서에서 삼성전자가 자체 개발한 생성형 AI의 이름에 대한 정보를 찾을 수 없습니다.'`*

    * **`chunk_size` = `7` → 검색 결과 ⭕️**
      * *`"삼성전자가 자체 개발한 생성형 AI의 이름은 '삼성 가우스'입니다."`*

  * [**`myrag4.py`**](../15_Evaluations/myrag4.py)로 정리

* d.
  * **`잦은 커널 충돌로 인한 문제` → `python` 파일로 실행 → 터미널에서 직접 확인하기**

  * [**`eval_script.py`**](../15_Evaluations/eval_script.py) 실행 → 터미널에서 결과 바로 확인

  * ![`RAG_EVAL_4`](../15_Evaluations/assets/RAG_EVAL_4.png)

<small>

```bash

      (가상환경)/15_Evaluations/eval_script.py

      🚀 RAG 시스템 초기화 시작...

      📂 스크립트 위치: ... /15_Evaluations

      📄 PDF 경로: ... /data/SPRI_AI_Brief_2023년12월호_F.pdf

      ✅ 파일 확인 완료!

      ✅ LLM 생성 완료: gemini-2.5-flash-lite

      ✅ PDF 로드 완료: 23 페이지

      ✅ 청크 분할 완료: 119 청크 (크기=300, 오버랩=50)


      ✅ LLM 생성 완료: gemini-2.5-flash-lite

      ✅ 청크 분할 완료: 119 청크 (크기=300, 오버랩=50)
      pytorch_model.bin: 100%|████████████████████████████████████████████████████████████████████| 2.27G/2.27G [00:03<00:00, 695MB/s]
      tokenizer_config.json: 100%|███████████████████████████████████████████████████████████████████| 444/444 [00:00<00:00, 1.92MB/s]
      sentencepiece.bpe.model: 100%|██████████████████████████████████████████████████████████████| 5.07M/5.07M [00:06<00:00, 781kB/s]
      tokenizer.json: 100%|██████████████████████████████████████████████████████████████████████| 17.1M/17.1M [00:06<00:00, 2.70MB/s]
      special_tokens_map.json: 100%|█████████████████████████████████████████████████████████████████| 964/964 [00:00<00:00, 3.61MB/s]
      config.json: 100%|██████████████████████████████████████████████████████████████████████████████| 191/191 [00:00<00:00, 628kB/s]

      ✅ 임베딩 모델 로드 완료: BAAI/bge-m3                                                                 | 0.00/191 [00:00<?, ?B/s]
      model.safetensors:   0%|                                                                  | 2.19M/2.27G [00:24<1:26:03, 439kB/s]✅ 벡터스토어 생성 완료: FAISS

      ✅ 검색기 생성 완료 (k=7, search_type=similarity)

      ✅ RAG 체인 생성 완료

      ==================================================
      🧪 테스트 실행...
      ==================================================

      View the evaluation results for experiment: 'RAG_EVAL_K7-3a35bd45' at:
      https://smith.langchain.com/o/2c3342d3-1170-4ffa-86fd-f621199e0b9c/datasets/420dd308-2ebd-44c9-8ce8-9aff3886dc8e/compare?selectedSessions=cdb9ea10-e3c1-47bc-bfe1-48f651364c28


      5it [00:14,  2.90s/it]
      5it [00:14,  2.18s/it]

      ==================================================
      ✅ 평가 완료!
      ==================================================

      결과: <ExperimentResults RAG_EVAL_K7-3a35bd45>

      model.safetensors: 100%|███████████████████████████████████████████████████████████████████| 2.27G/2.27G [02:14<00:00, 16.9MB/s]

```

---

* **`현재 상황 분석`**

  | 항목 | 개수 | 정확도 |
  |------|------|--------|
  | **CORRECT** | 3/5 | 60% |
  | **INCORRECT** | 2/5 | 40% |

<br>

* 진전
  * 이전: 1/5 (20%)
  * 현재: 3/5 (60%)
  * **향상: +200%!**

<br>

* **`RAG` 평가**

  * 어려움

    | 이유 | 설명 |
    |------|------|
    | **초기 정확도** | 20-40%는 정상 |
    | **반복 개선** | 60% → 80% → 90%로 점진적 향상 |
    | **완벽은 없음** | 100%는 거의 불가능 |

<br>

* 

  * 현재 단계

    ```markdown
        일반적인 RAG 평가 발전:

        1단계: 20-40% (초기)
        2단계: 50-60% (개선 후)  ←  여기!
        3단계: 70-80% (최적화)
        4단계: 80-90% (완성)
    ```

<br>

* **`향상 방법`**

  * **a. `k값 증가`**

  * **b. `chunk_size` 조정하기**

  * **c. `프롬프트 개선하기`**

  * **d. `다른 임베딩 모델 시도하기`**

---

#### **3) `LangSmith 테스트를 위한 Dataset 생성`**

* **`Dataset & Testing`에 새로운 데이터셋 생성하기**

<small>

  * ![langsmith에 새로운 데이터셋 생성하기](../15_Evaluations/assets/eval-06.png)

* `csv` 파일에서 `LangSmith UI` 사용 → 직접 데이터셋 생성 가능

* *참고: [`LangSmith UI 문서`](https://docs.langchain.com/langsmith/manage-datasets)*

In [ ]:
from langsmith import Client

client = Client()
dataset_name = "RAG_EVAL_DATASET"

In [ ]:
# 데이터셋 생성 함수

def create_dataset(client, dataset_name, description=None):
    for dataset in client.list_datasets():
        if dataset.name == dataset_name:
            return dataset

    dataset = client.create_dataset(
        dataset_name=dataset_name,
        description=description,
    )
    return dataset

In [ ]:
# 데이터셋 생성
dataset = create_dataset(client, dataset_name)          # 2.9s

In [ ]:
# 생성된 데이터셋에 예제 추가

client.create_examples(
    inputs=[{"question": q} for q in df["question"].tolist()],
    outputs=[{"answer": a} for a in df["answer"].tolist()],
    dataset_id=dataset.id,
)

<small>

* **`client.creat_examples() 추가하기`** - (`0.1s`)

    ```python
    {'example_ids': ['8d34ad4e-e3be-47ca-9a6a-9929608b60ca',
    '71841ffc-cfbe-4e05-b0b0-ef85e8a6ebd4',
    'e0d645a9-6d97-41da-8d45-cc5e8d6f0a37'],
    'count': 3}
    ```

* **`데이터셋에 예제 나중에 추가 가능`**

In [ ]:
# 새로운 질문 목록
new_questions = [
    "삼성전자가 만든 생성형 AI의 이름은 무엇인가요?",
    "구글이 테디노트에게 20억달러를 투자한 것이 사실입니까?",
]

In [ ]:
# 새로운 답변 목록
new_answers = [
    "삼성전자가 만든 생성형 AI의 이름은 테디노트 입니다.",
    "사실이 아닙니다. 구글은 앤스로픽에 최대 20억 달러를 투자하기로 합의했으며, 이 중 5억 달러를 우선 투자하고 향후 15억 달러를 추가로 투자하기로 했습니다.",
]

In [ ]:
# UI에서 업데이트된 버전 확인

client.create_examples(
    inputs=[{"question": q} for q in new_questions],
    outputs=[{"answer": a} for a in new_answers],
    dataset_id=dataset.id,
)

<small>

* **`UI에서 업데이트된 버전 확인하기`** - (`0.1s`)

    ```python
    {'example_ids': ['7080404f-878d-4e22-9256-70a5d1b86ab3',
    'b8599d26-9990-4153-8a92-0dc779087f2c'],
    'count': 2}
    ```

* **✓ `데이터셋 준비 완료됨`**

---

* next: ***`05. LLM-as-Judge`***

---